In [3]:
import sys
sys.path.append('..')
import igraph as ig
import multiprocessing as mp
from pyzx.drawing import *

In [4]:
class ZXGraph():
    def __init__(self):
        self.ig = ig.Graph()
    def set_vtype(self, v, t):
        if isinstance(v, ig.Vertex):
            v['t'] = t
        else:
            self.ig.vs[v]['t'] = t
    def vtype(self, v):
        if isinstance(v, ig.Vertex):
            return v['t']
        else:
            return self.ig.vs[v]['t']
    def add_vertices(self, n):
        self.ig.add_vertices(n)

def match_bialg(g, interior=False):
    for e in g.es:
        v0 = e.source
        v1 = e.target
        v0t = g.vs[v0]['t']
        v1t = g.vs[v1]['t']
        if ((v0t == 1 and v1t == 2) or (v0t == 2 and v1t == 1)):
            if (
                not interior or (
                all([n['t'] == v1t for n in g.vs[v0].neighbors()]) and
                all([n['t'] == v0t for n in g.vs[v1].neighbors()]))
            ):
                return [v0,v1]
    return None

def match_bialg_parallel(g, num=100):
    candidates = set(range(len(g.es)))
    #return candidates
    i = 0
    m = []
    while (num == -1 or i < num) and len(candidates) > 0:
        e = g.es[candidates.pop()]
        #if i >= num: break
        v0 = e.source
        v1 = e.target
        #if v0 in inv or v1 in inv: continue
        v0t = g.vs[v0]['t']
        v1t = g.vs[v1]['t']
        if ((v0t == 1 and v1t == 2) or (v0t == 2 and v1t == 1)):
            v0n = [n for n in g.vs[v0].neighbors() if n.index != v1]
            v1n = [n for n in g.vs[v1].neighbors() if n.index != v0]
            if (
                all([n['t'] == v1t for n in v0n]) and
                all([n['t'] == v0t for n in v1n])):
                i += 1
                for v in v0n:
                    for c in g.incident(v, mode=ig.ALL): candidates.discard(c)
                for v in v1n:
                    for c in g.incident(v, mode=ig.ALL): candidates.discard(c)
                v0n = [v.index for v in v0n]
                v1n = [v.index for v in v1n]
                m.append([v0,v1,v0n,v1n])
    return m
    
def bialg_parallel(g, matches):
    dv = []
    ae = []
    de = []
    for m in matches:
        dv.append(m[0])
        dv.append(m[1])
        es = [(i,j) for i in m[2] for j in m[3]]
        for e in es:
            if g.are_connected(e[0], e[1]): de.append(e)
            else: ae.append(e)
    
    g.delete_edges(de)
    g.add_edges(ae)
    g.delete_vertices(dv)
    g.vs.select(_degree=0).delete()
    
def bialg(g, match, check=False):
    v0 = match[0]
    v1 = match[1]
    v0t = g.vs[v0]['t']
    v1t = g.vs[v1]['t']

    if check:
        if not (
            g.are_connected(v0,v1) and
            ((v0t == 1 and v1t == 2) or
            (v0t == 2 and v1t == 1))
        ): return False
    
    n0 = [n for n in g.vs[v0].neighbors() if n.index != v1]
    n1 = [n for n in g.vs[v1].neighbors() if n.index != v0]
    
    # add dummy nodes around v0, v1 as necessary.
    for i in range(len(n0)):
        if (n0[i]['t'] != v1t):
            g.add_vertex()
            newv = g.vs[len(g.vs)-1]
            newv['t'] = v1t
            g.delete_edges([(v0,n0[i].index)])
            g.add_edges([(n0[i].index, newv.index), (newv.index, v0)])
            n0[i] = newv
    
    for i in range(len(n1)):
        if (n1[i]['t'] != v0t):
            g.add_vertex()
            newv = g.vs[len(g.vs)-1]
            newv['t'] = v0t
            g.delete_edges([(v1,n1[i].index)])
            g.add_edges([(v1,newv.index),(newv.index,n1[i].index)])
            n1[i] = newv
    
    for s in n0:
        for t in n1:
            if g.are_connected(s,t): g.delete_edges([(s,t)])
            else: g.add_edge(s,t)
    
    
    # delete vertices at the end so we don't mess up indices
    g.delete_vertices([v0,v1] + [v for v in n0 + n1 if v.degree() < 2])
    return True

In [36]:
g = zigzag(10)
g.es[4].source = 5

AttributeError: attribute 'source' of 'igraph.Edge' objects is not writable

In [26]:
g = ZXGraph()
g.add_vertices(200)
g.set_vtype(g.ig.vs[10], 1)
g.vtype(10)

1

In [6]:
def zigzag(sz):
    g = ig.Graph()
    g.add_vertex(t=None,d=None)
    g.add_vertices(2*sz+3)
    for i in range(1,sz+1):
        g.vs[2*i]['t'] = (i%2)+1
        g.vs[2*i+1]['t'] = (i%2)+1
    g.add_edges([(0,2),(1,3)])
    g.add_edges([(2*i,2*i+2) for i in range(1,sz)])
    g.add_edges([(2*i,2*i+3) for i in range(1,sz)])
    g.add_edges([(2*i+1,2*i+2) for i in range(1,sz)])
    g.add_edges([(2*i+1,2*i+3) for i in range(1,sz)])
    g.add_edges([(2*sz,2*sz+2),(2*sz+1,2*sz+3)])
    return g

In [5]:
%time g = zigzag(1000000)
%time m = match_bialg(g, interior=True)
%time bialg(g, m)

CPU times: user 6.16 s, sys: 2.19 s, total: 8.34 s
Wall time: 8.35 s
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.77 ms
CPU times: user 5.72 s, sys: 1.06 s, total: 6.78 s
Wall time: 6.91 s


True

In [12]:
g1 = zigzag(2000)
def f():
    while True:
        m = match_bialg(g1, interior=True)
        if m: bialg(g1, m)
        else: break

%time f()

CPU times: user 2.06 s, sys: 0 ns, total: 2.06 s
Wall time: 2.06 s


In [7]:
%time g = zigzag(10**7)

CPU times: user 29.8 s, sys: 15 s, total: 44.8 s
Wall time: 44.8 s


In [30]:
%time g = zigzag(5 * 10**5)
%time m = match_bialg_parallel(g, num=-1)
%time bialg_parallel(g, m)

CPU times: user 1.48 s, sys: 359 ms, total: 1.84 s
Wall time: 1.82 s
CPU times: user 3.62 s, sys: 188 ms, total: 3.81 s
Wall time: 3.84 s
CPU times: user 1.62 s, sys: 234 ms, total: 1.86 s
Wall time: 1.86 s


In [39]:
g = zigzag(20)
match_bialg_parallel(g)

[[4, 6, [2, 3, 7], [5, 8, 9]],
 [10, 12, [8, 9, 13], [11, 14, 15]],
 [16, 18, [14, 15, 19], [17, 20, 21]],
 [22, 24, [20, 21, 25], [23, 26, 27]],
 [28, 30, [26, 27, 31], [29, 32, 33]],
 [34, 36, [32, 33, 37], [35, 38, 39]]]

In [10]:
%time g = zigzag(100000)
def f():
    print("normalising ZX diagram with " + str(len(g.vs)) +
          " vertices and " + str(len(g.es)) + " edges")
    it = 0
    while True:
        it += 1
        m = match_bialg_parallel(g, num=-1)
        print("got " + str(len(m)) + " matches of bialgebra")
        if len(m) != 0: bialg_parallel(g, m)
        else: break
    print("completed in " + str(it) + " iterations")

%time f()

CPU times: user 609 ms, sys: 62.5 ms, total: 672 ms
Wall time: 664 ms
normalising ZX diagram with 200004 vertices and 400000 edges
got 33333 matches of bialgebra
got 11111 matches of bialgebra
got 3703 matches of bialgebra
got 1235 matches of bialgebra
got 411 matches of bialgebra
got 137 matches of bialgebra
got 46 matches of bialgebra
got 15 matches of bialgebra
got 5 matches of bialgebra
got 2 matches of bialgebra
got 1 matches of bialgebra
got 0 matches of bialgebra
completed in 12 iterations
CPU times: user 2.91 s, sys: 125 ms, total: 3.03 s
Wall time: 3.09 s


In [33]:
g.vs[4]['t']

In [9]:
g = ig.Graph()
%time g.add_vertices(10000000)
%time g.delete_vertices([9999])

Wall time: 61.4 ms
Wall time: 251 ms


In [1]:
%time g = zigzag(1000000)

NameError: name 'zigzag' is not defined